In [1]:
import sympy as sm
from sympy import diff, Symbol, cos,diag, Rational
from scipy.integrate import solve_ivp,OdeSolver,odeint
import numpy as np
from sympy.physics.mechanics import dynamicsymbols
import math
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement
from functools import reduce

In [2]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex="mathjax", pretty_print=False)

In [3]:
m1, m2, l1, l2, g ,tau1, tau2,tau3,tau4, c1, c2, c3, c4, L, r1,r2= sm.symbols('m1 m2 l1 l2 g tau1 tau2 tau3 tau4 c1 c2 c3 c4 L r1 r2', real =True)
t = Symbol('t')
theta1, theta2, phi1, phi2 =  dynamicsymbols('theta1 theta2 phi1 phi2')

theta1d = dynamicsymbols('theta1',1)
theta2d = dynamicsymbols('theta2',1)
phi1d = dynamicsymbols('phi1',1)
phi2d = dynamicsymbols('phi2',1)
theta1dd = dynamicsymbols('theta1',2)
theta2dd = dynamicsymbols('theta2',2)
phi1dd = dynamicsymbols('phi1',2)
phi2dd = dynamicsymbols('phi2',2)
g, m1, m2, l1, l2, theta1, theta2, phi1, phi2, theta1d, theta2d, phi1d,phi2d, theta1dd, theta2dd, phi1dd,phi2dd, tau1, tau2,tau3,tau4,r1,r2

(g, m1, m2, l1, l2, theta1, theta2, phi1, phi2, theta1', theta2', phi1', phi2', theta1'', theta2'', phi1'', phi2'', tau1, tau2, tau3, tau4, r1, r2)

In [4]:
x1_ = 0.5*l1*sm.cos(theta1)*sm.sin(phi1)
y1_ = 0.5*l1*sm.sin(theta1)*sm.sin(phi1)
z1_ = 0.5*l1*sm.cos(phi1)

x2_ = l1*sm.cos(theta1)*sm.sin(phi1) + 0.5*l2*sm.cos(theta2)*sm.sin(phi2)
y2_ = l1*sm.sin(theta1)*sm.sin(phi1) + 0.5*l2*sm.sin(theta2)*sm.sin(phi2)
z2_ = l1*sm.cos(phi1) + 0.5*l2*cos(phi2 )

x1_dot = sm.diff(x1_,t)
y1_dot = sm.diff(y1_,t)
z1_dot = sm.diff(z1_,t)

x2_dot = sm.diff(x2_,t)
y2_dot = sm.diff(y2_,t)
z2_dot = sm.diff(z2_,t)

In [5]:
I1 = Rational('1/12')*m1*l1**2 + Rational('1/4')*m1*r1**2

In [6]:
v1 = Rational('0.5')*l1*theta1d

In [7]:
w1 = theta1d

In [8]:
k1 = Rational('1/2')*m1*v1**2 + Rational('1/2')*I1*w1**2

In [9]:
v2 = x2_dot**2 + y2_dot**2 + z2_dot**2

In [10]:
w2 = theta1d + theta2d

In [11]:
I2 = Rational('1/12')*m2*l2**2 + Rational('1/4')*m2*r2**2

In [12]:
k2 = Rational('1/2')*m2*v2**2 + Rational('1/2')*I2*w2**2

In [13]:
p1 = m1*g*z1_

In [14]:
p2 = m2*g*z2_

In [16]:
#kinetic
ke = k1 + k2

#potential
pe = p1 + p2

# the lagrangian equation
Ls = ke - pe

In [17]:
f1 = (sm.diff(sm.diff(Ls,theta1d),t) - sm.diff(Ls,theta1)).expand()
f2 = (sm.diff(sm.diff(Ls,theta1d),t) - sm.diff(Ls,theta2)).expand()
f3 = (sm.diff(sm.diff(Ls,phi1d),t) - sm.diff(Ls,phi1)).expand()
f4 = (sm.diff(sm.diff(Ls,phi2d),t) - sm.diff(Ls,phi2)).expand()

In [19]:
#inertia term
rows = []
for f in [f1,f2,f3,f4]:
    columns = []
    for v in [theta1dd,theta2dd,phi1dd,phi2dd]:
        columns.append(f.coeff(v))
    rows.append(columns)

M = sm.Matrix(rows)

In [20]:
#centrifugal / Coriolis force term,
rows = []
coordinates = list(combinations_with_replacement([theta1d,theta2d,phi1d,phi2d], 2))
c_v = [sm.prod(i) for i in coordinates]
for f in [f1,f2,f3,f4]:
    columns = []
    for v in c_v:
        columns.append(f.coeff(v)*v)
    rows.append(sum(columns))

C = sm.Matrix(rows)

In [21]:
# gravity term
rows = []
for f in [f1,f2,f3,f4]:
    columns = []
    for v in [theta1dd,theta2dd,phi1dd,phi2dd]:
        columns.append(f.coeff(v)*v)
    rows.append(sum(columns))

G = sm.Matrix([f1,f2,f3,f4]) - sm.Matrix(rows) - C

In [22]:
tau = sm.Matrix([tau1, tau2, tau3, tau4])
tau

Matrix([
[tau1],
[tau2],
[tau3],
[tau4]])